In [3]:
import os
import dspy
import csv

In [4]:
import ujson

In [5]:
from sentence_transformers import SentenceTransformer

In [8]:
import pandas as pd

### ref: https://dspy.ai/tutorials/rag/

In [6]:
def InitializeLLM():
    local_config = {
        "api_base": "http://localhost:11434/v1",  # 注意需加/v1路徑
        "api_key": "NULL",  # 特殊標記用於跳過驗證
        "model": "deepseek-r1:7b",
        "custom_llm_provider":"deepseek"
    }
    dspy.configure(
        lm=dspy.LM(
            **local_config
        )
    )

In [7]:
InitializeLLM();

In [9]:
def llm_predict(queryStr:str=None):
    if queryStr == None:
        raise ValueError("query string is none, please input query string.")
    promptPatternStr = "question -> answer"
    qa = dspy.Predict(promptPatternStr);
    response = qa(question=queryStr);
    print(f"llm:{response.answer}");

In [10]:
llm_predict("what day is today")

llm:October 26th


In [12]:
dspy.inspect_history(n=1)





[2025-02-21T16:17:14.651093]

System message:

Your input fields are:
1. `question` (str)

Your output fields are:
1. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## question ## ]]
{question}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `question`, produce the fields `answer`.


User message:

[[ ## question ## ]]
what day is today

Respond with the corresponding output fields, starting with the field `[[ ## answer ## ]]`, and then ending with the marker for `[[ ## completed ## ]]`.


Response:

[[ ## answer ## ]]
October 26th

[[ ## completed ## ]]







In [13]:
cot = dspy.ChainOfThought('question -> response')
cot(question="should curly braces appear on their own line?")

Prediction(
    reasoning='{The reasoning field is where you would provide a detailed thought process or explanation based on the input question. It should be structured in a way that clearly shows how you arrived at your answer.}',
    response='{The response field is where you provide a concise and direct answer to the question, typically one or two sentences long. It should be clear and to the point, summarizing the key points from your reasoning.}'
)

In [20]:
# with open("./data/ragqa_arena_tech_examples.jsonl") as f:
#     data = [ujson.loads(line) for line in f]

### Basic RAG
- dspy.Embedder: https://dspy.ai/api/models/Embedder/
- multihop Search: https://dspy.ai/tutorials/multihop_search/?h=search

In [11]:
max_characters = 6000  # for truncating >99th percentile of documents
topk_docs_to_retrieve = 5  # number of documents to retrieve per search query
qKey = "Question"
qAns = "Answer"
df = pd.read_csv("../prompt_augment_data/data/qa.csv")
df_clean = df.groupby(qKey)[qAns].apply(lambda x: '\n'.join(x.unique())).reset_index()
df_clean[qAns] = df_clean[qAns].str.replace(r'(\w+)\s*=\s*(\d+)', r'\1=\2', regex=True)
answers = df_clean[qAns].str.strip().tolist()
print(f"Loaded {len(answers)} documents. Will encode them below.")
model = SentenceTransformer("sentence-transformers/static-retrieval-mrl-en-v1", device="cpu")
embedder = dspy.Embedder(model=model.encode, dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=answers, k=topk_docs_to_retrieve)

Loaded 145 documents. Will encode them below.


In [12]:
answers[1]

'PPT Control set manual, if the parameters below remain 0, they are disabled.\nThermal Control set manual'

In [13]:
class RAG(dspy.Module):
    def __init__(self):
        self.respond=dspy.ChainOfThought('context, question -> answer');

    def forward(self,question):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [ ]:
rag = RAG();

In [15]:
print(rag(question="how to fix hang up 0xabe4?"));

Prediction(
    reasoning='The problem with code address 0xabe4 is likely related to system reboot times. The patch suggests disabling gEfiAmdAgesaPkgTokenSpaceGuid.PcdAcpController, which can help fix long reboot times that may be causing the hangup at this specific address.',
    answer='To fix the hangup issue at code address 0xabe4:\n1. Locate the system configuration file where `gEfiAmdAgesaPkgTokenSpaceGuid.PcdAcpController` is defined.\n2. Set its value to `FALSE`.\n3. Apply this change to resolve the hangup problem.'
)


In [18]:
print(rag(question="aquarius password behavior customizing"))

Prediction(
    reasoning='The question asks about customizing the Aquarius password behavior. The context provided includes a file named "Aquarius Password behavior.zip," which is likely related to this customization.',
    answer='To customize the Aquarius password behavior, you can refer to the patch file `Aquarius Password behavior.zip`. This file contains the necessary configurations or scripts needed for the customization process.'
)


In [19]:
print(rag(question="aquarius密碼客製化行為"))

Prediction(
    reasoning='The context provided mentions issues related to hardware configurations and APU (Accelerated Processing Unit) settings, specifically regarding power limits when certain PPT APU only values are set. However, the question pertains to "aquarius passwords customization behavior," which is not directly addressed in the given context.',
    answer='To address password customization behavior for Aquarius, additional information or specific details about the system configuration and intended modifications would be required. Please provide more context or clarify your requirements so that I can assist you effectively.'
)


In [20]:
print(rag(question="BSOD"))

Prediction(
    reasoning='The context indicates that there are existing solutions related to fixing BSOD issues with D3Hot and NVMe HDDs. However, since these attempts have not resolved the issue, it suggests exploring alternative configurations or additional patches.',
    answer='To address the BSOD prevention, refer to the patch file `NmiBsodStudy.zip` for further troubleshooting steps.'
)


In [21]:
dspy.inspect_history()





[2025-02-25T12:12:51.605656]

System message:

Your input fields are:
1. `context` (str)
2. `question` (str)

Your output fields are:
1. `reasoning` (str)
2. `answer` (str)

All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## answer ## ]]
{answer}

[[ ## completed ## ]]

In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `answer`.


User message:

[[ ## context ## ]]
[1] «Revise the code to configure the Flash BIOS update sequence to prevent BSOD issues occurring under the operating system.»
[2] «Revise the code to configure the Flash BIOS update sequence to prevent BSOD issues occurring under the operating system.»
[3] «Refer to patch: NVMe HDD password for D3Hot fix bsod critical process died.7z»
[4] «««
    This is caused by the NVMe drive not supporting 
    D3Hot